In [4]:
print("Installing necessary libraries...")
!pip install openai requests -q

print("Importing modules...")
import openai
import requests
import json
import os
import sqlite3
import time
from google.colab import userdata, drive

print("Ready for the next step!")


Installing necessary libraries...
Importing modules...
Ready for the next step!


In [6]:
# === 2. Setup API Keys and Google Drive ===
print("Mounting Google Drive...")
drive.mount("/content/drive")

print("Loading API keys from Colab Secrets...")
try:
    OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
    SPOONACULAR_API_KEY = userdata.get("SPOONACULAR_API_KEY")
    OPENWEATHERMAP_API_KEY = userdata.get("OPENWEATHERMAP_API_KEY")

    # Check if keys are loaded
    if not all([OPENAI_API_KEY, SPOONACULAR_API_KEY, OPENWEATHERMAP_API_KEY]):
        raise ValueError("Not all API keys found in Colab Secrets. Please add OPENAI_API_KEY, SPOONACULAR_API_KEY, OPENWEATHERMAP_API_KEY.")

    openai.api_key = OPENAI_API_KEY
    print("API keys loaded successfully.")

except Exception as e:
    print(f"Error loading keys or mounting drive: {e}")
    print("Please ensure you have added the keys to Colab Secrets and authorized Google Drive access.")

# Define the path to the database file on your Google Drive
# You can change the MyDrive/ColabData folder to any other
DB_FOLDER = "/content/drive/MyDrive/ColabData"
if not os.path.exists(DB_FOLDER):
    os.makedirs(DB_FOLDER)
DB_PATH = os.path.join(DB_FOLDER, "recipe_assistant_db.sqlite")
print(f"Database will be stored at: {DB_PATH}")


Mounting Google Drive...
Mounted at /content/drive
Loading API keys from Colab Secrets...
API keys loaded successfully.
Database will be stored at: /content/drive/MyDrive/ColabData/recipe_assistant_db.sqlite


In [7]:
# === 3. Functions for Working with SQLite Database ===

def get_db_connection():
    """Establishes a connection to the SQLite DB."""
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row # Return rows as dictionaries
    return conn

def init_db():
    """Initializes the DB: creates the table if it doesn't exist."""
    conn = None # Initialize conn
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        # Fixed line breaks in DEFAULT
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS user_profiles (
                user_id TEXT PRIMARY KEY,
                preferences_json TEXT DEFAULT 	'{}',
                allergies_json TEXT DEFAULT 	'[]'
            );
        """)
        conn.commit()
        print("Database initialized successfully.")
    except Exception as e:
        print(f"Error initializing DB: {e}")
    finally:
        if conn:
            conn.close()

def get_user_profile(user_id):
    """Gets the user profile (preferences and allergies) from the DB."""
    conn = None
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT preferences_json, allergies_json FROM user_profiles WHERE user_id = ?", (user_id,))
        row = cursor.fetchone()
        if row:
            preferences = json.loads(row["preferences_json"])
            allergies = json.loads(row["allergies_json"])
            return json.dumps({"user_id": user_id, "preferences": preferences, "allergies": allergies})
        else:
            # If the user doesn't exist, create an empty profile
            cursor.execute("INSERT INTO user_profiles (user_id) VALUES (?)", (user_id,))
            conn.commit()
            return json.dumps({"user_id": user_id, "preferences": {}, "allergies": [], "message": "New profile created."})
    except Exception as e:
        print(f"Error in get_user_profile for {user_id}: {e}")
        return json.dumps({"error": f"Error retrieving user profile for {user_id}"})
    finally:
        if conn:
            conn.close()

def save_user_preference(user_id, preference_type, value, action="add"):
    """Adds or removes a user preference (like/dislike)."""
    conn = None
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT preferences_json FROM user_profiles WHERE user_id = ?", (user_id,))
        row = cursor.fetchone()
        if not row:
            # Important: Close the connection before returning the error
            if conn: conn.close()
            return json.dumps({"error": f"User {user_id} not found. Profile was not initialized."}) # Should be created via get_user_profile first

        preferences = json.loads(row["preferences_json"])
        if preference_type not in preferences:
            preferences[preference_type] = []

        item_list = preferences[preference_type]
        value_lower = value.lower() # Store in lowercase

        if action == "add":
            if value_lower not in item_list:
                item_list.append(value_lower)
                # Fixed line breaks in f-string
                message = f"Added 	'{value}' to 	'{preference_type}'."
            else:
                message = f"	'{value}' is already in 	'{preference_type}'."
        elif action == "remove":
            if value_lower in item_list:
                item_list.remove(value_lower)
                message = f"Removed 	'{value}' from 	'{preference_type}'."
            else:
                message = f"	'{value}' not found in 	'{preference_type}'."
        else:
             # Important: Close the connection before returning the error
             if conn: conn.close()
             return json.dumps({"error": "Invalid action, use 'add' or 'remove'."})

        cursor.execute("UPDATE user_profiles SET preferences_json = ? WHERE user_id = ?", (json.dumps(preferences), user_id))
        conn.commit()
        return json.dumps({"user_id": user_id, "preferences": preferences, "message": message})

    except Exception as e:
        print(f"Error in save_user_preference for {user_id}: {e}")
        return json.dumps({"error": f"Error saving preference for {user_id}"})
    finally:
        if conn:
            conn.close()

def save_user_allergy(user_id, allergy_value, action="add"):
    """Adds or removes a user allergy."""
    conn = None
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT allergies_json FROM user_profiles WHERE user_id = ?", (user_id,))
        row = cursor.fetchone()
        if not row:
            # Important: Close the connection before returning the error
            if conn: conn.close()
            return json.dumps({"error": f"User {user_id} not found. Profile was not initialized."}) # Should be created via get_user_profile first

        allergies = json.loads(row["allergies_json"])
        value_lower = allergy_value.lower()

        if action == "add":
            if value_lower not in allergies:
                allergies.append(value_lower)
                # Fixed line breaks in f-string
                message = f"Allergy added: 	'{allergy_value}'."
            else:
                message = f"Allergy 	'{allergy_value}' already recorded."
        elif action == "remove":
            if value_lower in allergies:
                allergies.remove(value_lower)
                message = f"Allergy removed: 	'{allergy_value}'."
            else:
                message = f"Allergy 	'{allergy_value}' not found."
        else:
            # Important: Close the connection before returning the error
            if conn: conn.close()
            return json.dumps({"error": "Invalid action, use 'add' or 'remove'."})

        cursor.execute("UPDATE user_profiles SET allergies_json = ? WHERE user_id = ?", (json.dumps(allergies), user_id))
        conn.commit()
        return json.dumps({"user_id": user_id, "allergies": allergies, "message": message})

    except Exception as e:
        print(f"Error in save_user_allergy for {user_id}: {e}")
        return json.dumps({"error": f"Error saving allergy for {user_id}"})
    finally:
        if conn:
            conn.close()

# Call DB initialization when the cell runs
init_db()
print("DB functions defined.")


Database initialized successfully.
DB functions defined.


In [8]:
# === 4. Functions for External APIs (Weather, Recipes) ===

def get_weather(city):
    """Gets the current weather for the specified city from OpenWeatherMap."""
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    # Using f-string for cleaner URL construction
    complete_url = f"{base_url}appid={OPENWEATHERMAP_API_KEY}&q={city}&units=metric&lang=en" # Changed lang to en
    try:
        response = requests.get(complete_url)
        response.raise_for_status() # Check for HTTP errors
        data = response.json()
        if data.get("cod") != 404 and data.get("cod") != "404": # Check for numeric and string 404
            main = data.get("main", {})
            weather = data.get("weather", [{}])[0]
            temp = main.get("temp")
            feels_like = main.get("feels_like")
            description = weather.get("description")
            return json.dumps({
                "city": city,
                "temperature": temp,
                "feels_like": feels_like,
                "description": description
            })
        else:
            return json.dumps({"error": "City not found"})
    except requests.exceptions.RequestException as e:
        print(f"OpenWeatherMap API error: {e}")
        return json.dumps({"error": f"Error requesting weather: {e}"})
    except Exception as e:
        print(f"Unexpected error in get_weather: {e}")
        return json.dumps({"error": "Unexpected error getting weather"})

def find_recipes(weather_description, temperature, user_query=None, cuisine=None, diet=None, intolerances=None, include_ingredients=None, exclude_ingredients=None, max_ready_time=None):
    """Searches for recipes on Spoonacular considering weather, user query, and other filters."""
    api_url = "https://api.spoonacular.com/recipes/complexSearch"
    params = {
        "apiKey": SPOONACULAR_API_KEY,
        "number": 3, # Return the top 3 options
        "addRecipeInformation": False # Don't request full info yet to save quota
    }

    # --- Smart logic for query formation based on weather and parameters ---
    # (IMPORTANT: This part needs refinement and improvement!)
    query_parts = [user_query] if user_query else []
    if temperature is not None:
        if temperature < 10:
            query_parts.append("warm soup stew bake comfort")
        elif temperature > 25:
            query_parts.append("cold salad light refreshing grill")
    # Changed "дождь" to "rain"
    if weather_description and "rain" in weather_description.lower():
        query_parts.append("indoor easy")

    effective_query = " ".join(filter(None, query_parts))
    if effective_query:
        params["query"] = effective_query

    # Add other filters if provided
    if cuisine: params["cuisine"] = cuisine
    if diet: params["diet"] = diet
    # Allergies are passed as a comma-separated string
    if intolerances: params["intolerances"] = ",".join(intolerances)
    if include_ingredients: params["includeIngredients"] = ",".join(include_ingredients)
    # Disliked products
    if exclude_ingredients: params["excludeIngredients"] = ",".join(exclude_ingredients)
    if max_ready_time: params["maxReadyTime"] = max_ready_time

    print(f"[DEBUG] Request to Spoonacular Complex Search with parameters: {params}")

    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()
        recipes = response.json().get("results", [])
        if not recipes:
            return json.dumps({"message": "No suitable recipes found based on your criteria."})
        # Return brief info: ID, title, image
        return json.dumps([{"id": r.get("id"), "title": r.get("title"), "image": r.get("image", "")} for r in recipes])
    except requests.exceptions.RequestException as e:
        print(f"Spoonacular API error (Complex Search): {e}")
        return json.dumps({"error": f"Error searching for recipes: {e}"})
    except Exception as e:
        print(f"Unexpected error in find_recipes: {e}")
        return json.dumps({"error": "Unexpected error searching for recipes"})

def get_recipe_details(recipe_id):
    """Gets recipe details by its ID from Spoonacular."""
    api_url = f"https://api.spoonacular.com/recipes/{recipe_id}/information"
    params = {"apiKey": SPOONACULAR_API_KEY, "includeNutrition": False}
    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()
        details = response.json()
        ingredients = [item.get("original") for item in details.get("extendedIngredients", [])]
        instructions = details.get("instructions", "Instructions not found.")
        # Can add instruction parsing into steps if needed
        return json.dumps({
            "title": details.get("title", "Untitled"),
            "ingredients": ingredients,
            "instructions": instructions,
            "sourceUrl": details.get("sourceUrl", "")
        })
    except requests.exceptions.RequestException as e:
        print(f"Spoonacular API error (Get Info): {e}")
        return json.dumps({"error": f"Error getting recipe details {recipe_id}: {e}"})
    except Exception as e:
        print(f"Unexpected error in get_recipe_details: {e}")
        return json.dumps({"error": f"Unexpected error getting recipe details {recipe_id}"})

print("Functions for external APIs defined.")


Functions for external APIs defined.


In [9]:
# === 5. Setup OpenAI Assistant ===

# EXPLICITLY PASS THE KEY WHEN CREATING THE CLIENT
try:
    client = openai.OpenAI(api_key=OPENAI_API_KEY)
    print("OpenAI client created successfully.")
except NameError:
    print("Error: OPENAI_API_KEY variable is not defined. Please run cell 2.")
    raise
except Exception as e:
    print(f"Unexpected error creating OpenAI client: {e}")
    raise

# Define the tools (functions) the assistant can call
tools_list = [
    # --- Weather Function ---
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather (temperature and description) for a specified city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string", "description": "The city to get the weather for, e.g., Dublin"}
                },
                "required": ["city"]
            }
        }
    },
    # --- Recipe Functions ---
    {
        "type": "function",
        "function": {
            "name": "find_recipes",
            "description": "Find recipes suitable for the current weather and user parameters (query, cuisine, diet, allergies, cooking time, include/exclude ingredients).",
            "parameters": {
                "type": "object",
                "properties": {
                    "weather_description": {"type": "string", "description": "Weather description (e.g., 'clear sky', 'light rain')"},
                    "temperature": {"type": ["number", "null"], "description": "Current temperature in Celsius (can be null if weather is unknown)"},
                    "user_query": {"type": "string", "description": "(Optional) Specific user query for a dish (e.g., 'chicken soup', 'something with salmon')"},
                    "cuisine": {"type": "string", "description": "(Optional) Desired cuisine (e.g., 'italian', 'mexican')"},
                    "diet": {"type": "string", "description": "(Optional) Dietary restrictions (e.g., 'vegetarian', 'gluten free')"},
                    "intolerances": {"type": "array", "items": {"type": "string"}, "description": "(Optional) List of products causing allergies or intolerances (e.g., ['nuts', 'dairy'])."},
                    "include_ingredients": {"type": "array", "items": {"type": "string"}, "description": "(Optional) List of desired ingredients.",},
                    "exclude_ingredients": {"type": "array", "items": {"type": "string"}, "description": "(Optional) List of ingredients to exclude (e.g., disliked products)."},
                    "max_ready_time": {"type": "integer", "description": "(Optional) Maximum cooking time in minutes.",}
                },
                "required": ["weather_description", "temperature"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_recipe_details",
            "description": "Get detailed information about a recipe (ingredients, preparation steps) by its ID.",
            "parameters": {
                "type": "object",
                "properties": {
                    "recipe_id": {"type": "integer", "description": "The ID of the recipe obtained from the find_recipes function"}
                },
                "required": ["recipe_id"]
            }
        }
    },
    # --- User Profile Functions ---
    {
        "type": "function",
        "function": {
            "name": "get_user_profile",
            "description": "Get saved preferences (likes/dislikes) and allergies for the specified user. If the profile doesn't exist, it will be created.",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_id": {"type": "string", "description": "Unique user identifier (e.g., username)."}
                },
                "required": ["user_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "save_user_preference",
            "description": "Save or remove a user preference (like/dislike) for a specific product or food type.",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_id": {"type": "string", "description": "User identifier.",},
                    "preference_type": {"type": "string", "enum": ["likes", "dislikes"], "description": "Preference type: 'likes' or 'dislikes'."},
                    "value": {"type": "string", "description": "Product or food type (e.g., 'chicken', 'mushrooms', 'spicy')."},
                    "action": {"type": "string", "enum": ["add", "remove"], "default": "add", "description": "Action: 'add' or 'remove'. Defaults to 'add'."}
                },
                "required": ["user_id", "preference_type", "value"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "save_user_allergy",
            "description": "Save or remove information about a user's allergy to a specific product.",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_id": {"type": "string", "description": "User identifier.",},
                    "allergy_value": {"type": "string", "description": "Product the user is allergic to (e.g., 'peanuts', 'milk', 'gluten')."},
                    "action": {"type": "string", "enum": ["add", "remove"], "default": "add", "description": "Action: 'add' or 'remove'. Defaults to 'add'."}
                },
                "required": ["user_id", "allergy_value"]
            }
        }
    },
]

# --- Assistant Instructions ---
# === Updated Assistant Instructions ===
ASSISTANT_INSTRUCTIONS = """
You are the Weather Recipe Guru, a friendly and creative culinary assistant. Your primary goal is to suggest recipes perfectly suited to the current weather and the user's personal preferences and restrictions. Aim for a natural and efficient conversation.

Initial Interaction Flow:
1.  **Greeting & Info Gathering:** Start with a friendly greeting. In your *first or second message*, politely ask for the user's name (for their profile, use it as user_id) AND the city they are in (to check the weather).
2.  **Profile & Weather:** Once you have the user_id and city:
    *   Call `get_user_profile` to load their saved preferences (likes/dislikes) and allergies. Remember these details.
    *   Call `get_weather` to get the current temperature and weather description.
3.  **Contextual Recipe Query:** Now that you have the profile (if any) and weather, ask the user what kind of meal they're looking for today. *Mention the weather* (e.g., "It's quite chilly in [City] today...") and *briefly acknowledge their known preferences/allergies* if available (e.g., "...and I know you like spicy food and avoid nuts...") to show you're using the context. Ask for their specific wishes (e.g., 'something quick', 'Italian cuisine', 'use chicken').

Recipe Suggestion Workflow:
4.  **Recipe Search:**
    a.  Analyze the weather (temperature, description) and the user's request.
    b.  Call the `find_recipes` function. Always pass `weather_description` and `temperature`. Also include:
        -   The user's specific request in `user_query`.
        -   Saved allergies in `intolerances`.
        -   Saved `dislikes` in `exclude_ingredients`.
        -   (Optional) Consider `likes` when forming `user_query` or pass them in `include_ingredients`.
        -   Any other user clarifications (cuisine, diet, cooking time).
    c.  Suggest 1-3 relevant recipes found (name and image URL if available).
5.  **Recipe Details:** If the user selects a recipe, call `get_recipe_details` using its ID. Present the name, ingredients list, and step-by-step instructions clearly.

Profile Management:
6.  **Saving Preferences/Allergies:** If the user explicitly states a preference or allergy (e.g., 'Remember, I don't like onions', 'I'm allergic to peanuts'), use their user_id to call `save_user_preference` (e.g., type='dislikes', value='onion') or `save_user_allergy` (e.g., allergy_value='peanut') respectively. Confirm briefly that you've saved it.

General Guidelines:
*   **Be Proactive but Polite:** Gather necessary information efficiently but maintain a friendly tone.
*   **Use Context:** Refer back to weather, preferences, and allergies when relevant.
*   **Clarify:** If a user request is ambiguous, ask clarifying questions.
"""


assistant = None # Initialize the assistant variable
try:
    # Check that the OpenAI client was successfully created earlier
    if 'client' not in locals() or client is None:
        print("Error: OpenAI client was not created. Please restart cell 5.") # Assuming this code is in cell 5, adjust if needed
        raise NameError("OpenAI client not initialized")

    assistant = client.beta.assistants.create(
        name="Weather Recipe Guru v2", # Translated name
        instructions=ASSISTANT_INSTRUCTIONS,
        model="gpt-4o-mini", # Using an economical model to start
        tools=tools_list,
    )
    print(f"Assistant created with ID: {assistant.id}")
except openai.AuthenticationError as auth_err:
    print(f"OpenAI authentication error: {auth_err}. Check your OpenAI API key in Colab Secrets.")
except Exception as e:
    print(f"Error creating assistant: {e}")
    # Attempt to find an existing assistant by name (if creation failed)
    try:
        print("Attempting to find an existing assistant...")
        my_assistants = client.beta.assistants.list(order="desc", limit="20")
        found = False # Flag indicating assistant found
        for existing_assistant in my_assistants.data:
            if existing_assistant.name == "Weather Recipe Guru v2": # Use translated name
                assistant = existing_assistant
                print(f"Found existing assistant with ID: {assistant.id}")
                found = True
                break
        if not found:
             print("Failed to create or find an assistant named 'Weather Recipe Guru v2'.")
    except Exception as list_e:
        print(f"Error searching for existing assistants: {list_e}")

# Add a check that assistant is not None before using
if assistant:
    print(f"Using assistant: {assistant.name} (ID: {assistant.id})")
else:
    print("\n!!! WARNING: Assistant was not created or found. Subsequent code will not work correctly. Check errors above and your OpenAI API key. !!!")



OpenAI client created successfully.
Assistant created with ID: asst_36JdysTVWsHUJHfz3DzIXloq
Using assistant: Weather Recipe Guru v2 (ID: asst_36JdysTVWsHUJHfz3DzIXloq)


In [10]:
# === 6. Assistant Interaction Loop ===

import json
import time

# --- Helper Functions for API Calls ---
# Note: These API call functions (get_weather, find_recipes, get_recipe_details)
# are defined here for clarity within the interaction loop cell.
# The user profile functions (get_user_profile, save_user_preference, save_user_allergy)
# are NOT redefined here; this cell will use the versions defined in cell 3 (SQLite).

def get_weather(city):
    """Gets weather from OpenWeatherMap API."""
    print(f"\n[Function] Requesting weather for city: {city}")
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': city,
        'appid': OPENWEATHERMAP_API_KEY, # Use key from cell 2
        'units': 'metric', # Celsius degrees
        'lang': 'en' # English language for description
    }
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status() # Check for HTTP errors
        data = response.json()

        # Extract necessary data
        temperature = data.get('main', {}).get('temp')
        description = data.get('weather', [{}])[0].get('description')

        if temperature is not None and description:
            result = {"temperature": temperature, "description": description}
            print(f"[Function] Weather received: {result}")
            return json.dumps(result) # Return JSON string
        else:
            print("[Function] Error: Failed to extract temperature or description from API response.")
            return json.dumps({"error": "Failed to get complete weather data."})

    except requests.exceptions.RequestException as e:
        print(f"[Function] Weather API error: {e}")
        return json.dumps({"error": f"Error requesting weather API: {e}"})
    except Exception as e:
        print(f"[Function] Unexpected error in get_weather: {e}")
        return json.dumps({"error": "Unexpected error getting weather."})

def find_recipes(weather_description, temperature, user_query=None, cuisine=None, diet=None, intolerances=None, include_ingredients=None, exclude_ingredients=None, max_ready_time=None):
    """Searches for recipes via Spoonacular API."""
    print(f"\n[Function] Searching recipes. Weather: {weather_description}, {temperature}°C. Query: {user_query}")
    base_url = "https://api.spoonacular.com/recipes/complexSearch"
    params = {
        'apiKey': SPOONACULAR_API_KEY, # Use key from cell 2
        'query': user_query if user_query else weather_description, # Use weather description as default query
        'cuisine': cuisine,
        'diet': diet,
        'intolerances': ','.join(intolerances)   if intolerances else None, # API expects comma-separated string
        'includeIngredients': ','.join(include_ingredients) if include_ingredients else None,
        'excludeIngredients': ','.join(exclude_ingredients) if exclude_ingredients else None,
        'maxReadyTime': max_ready_time,
        'number': 3, # Request 3 recipes for choice
        'addRecipeInformation': True # Include basic recipe info in response
    }

    # Add some logic based on weather for automatic filtering
    if temperature is not None:
        if temperature > 25: # Hot
            # Search for light dishes, salads, cold soups
            params['query'] = user_query if user_query else 'salad OR cold soup OR light meal'
            params['maxCalories'] = 600
        elif temperature < 5: # Cold
            # Search for warming dishes, soups, stews
            params['query'] = user_query if user_query else 'soup OR stew OR hot meal OR warming'
            params['minCalories'] = 400

    # Remove None parameters for a clean request
    params = {k: v for k, v in params.items() if v is not None}

    print(f"[Function] Parameters for Spoonacular request: {params}")

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        # Format result for the assistant
        recipes = []
        for recipe in data.get('results', []):
            recipes.append({
                "id": recipe.get('id'),
                "title": recipe.get('title'),
                "image": recipe.get('image'), # Image URL
                "readyInMinutes": recipe.get('readyInMinutes')
            })

        print(f"[Function] Recipes found: {len(recipes)}")
        if not recipes:
             print("[Function] No recipes found for the given criteria.")
             return json.dumps({"message": "Unfortunately, nothing was found for your request. Try changing the criteria."})
        return json.dumps(recipes) # Return JSON string with recipe list

    except requests.exceptions.RequestException as e:
        print(f"[Function] Recipe API error: {e}")
        # Attempt to extract error message from Spoonacular
        error_message = f"Error requesting recipe API: {e}"
        try:
            error_data = response.json()
            if 'message' in error_data:
                error_message += f" (API Message: {error_data['message']})"
        except: # Ignore errors parsing JSON error response
            pass
        return json.dumps({"error": error_message})
    except Exception as e:
        print(f"[Function] Unexpected error in find_recipes: {e}")
        return json.dumps({"error": "Unexpected error searching for recipes."})

def get_recipe_details(recipe_id):
    """Gets recipe details (ingredients, steps) by ID."""
    print(f"\n[Function] Requesting details for recipe ID: {recipe_id}")
    base_url = f"https://api.spoonacular.com/recipes/{recipe_id}/information"
    params = {
        'apiKey': SPOONACULAR_API_KEY,
        'includeNutrition': False # Don't request nutrition info to save quota
    }
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        # Extract and format necessary data
        details = {
            "title": data.get('title'),
            "readyInMinutes": data.get('readyInMinutes'),
            "servings": data.get('servings'),
            "ingredients": [item.get('original') for item in data.get('extendedIngredients', [])],
            "instructions": data.get('instructions'), # Can be HTML or text
            "sourceUrl": data.get('sourceUrl')
        }

        # Attempt to extract instruction steps if structured
        analyzed_instructions = data.get('analyzedInstructions', [])
        if analyzed_instructions and len(analyzed_instructions) > 0:
            steps = []
            for step_data in analyzed_instructions[0].get('steps', []):
                steps.append(f"{step_data.get('number')}. {step_data.get('step')}")
            if steps:
                details["instructions_steps"] = steps # Add separate field with step list

        print(f"[Function] Details for recipe '{details['title']}' received.")
        return json.dumps(details) # Return JSON string

    except requests.exceptions.RequestException as e:
        print(f"[Function] Recipe details API error: {e}")
        error_message = f"Error requesting recipe details API: {e}"
        try:
            error_data = response.json()
            if 'message' in error_data:
                error_message += f" (API Message: {error_data['message']})"
        except:
            pass
        return json.dumps({"error": error_message})
    except Exception as e:
        print(f"[Function] Unexpected error in get_recipe_details: {e}")
        return json.dumps({"error": "Unexpected error getting recipe details."})

# --- Main Interaction Loop ---

def run_assistant_interaction():
    # Check that the assistant was created in the previous cell
    if 'assistant' not in globals() or assistant is None:
        print("Error: Assistant not created or found. Please run cell 5.")
        return

    # Check for API keys
    if 'OPENWEATHERMAP_API_KEY' not in globals() or not OPENWEATHERMAP_API_KEY:
        print("Error: OpenWeatherMap API key not found. Please run cell 2.")
        return
    if 'SPOONACULAR_API_KEY' not in globals() or not SPOONACULAR_API_KEY:
        print("Error: Spoonacular API key not found. Please run cell 2.")
        return

    # Create a new dialogue (Thread) for this session
    try:
        thread = client.beta.threads.create()
        print(f"\nNew thread created with ID: {thread.id}")
    except Exception as e:
        print(f"Error creating thread: {e}")
        return

    # Dictionary to map function names to their actual implementations
    # These names will resolve to the functions defined in cell 3 (SQLite) and cell 6 (APIs)
    available_functions = {
        "get_weather": get_weather,
        "find_recipes": find_recipes,
        "get_recipe_details": get_recipe_details,
        "get_user_profile": get_user_profile, # Will use the one from cell 3
        "save_user_preference": save_user_preference, # Will use the one from cell 3
        "save_user_allergy": save_user_allergy, # Will use the one from cell 3
    }

    print("\n--- Weather Recipe Guru Ready --- ")
    print("Enter your message or 'exit' to finish.")

    while True:
        try:
            user_input = input("\nYou: ")
            if user_input.lower() == 'exit':
                print("\nGoodbye!")
                break

            # Send user message to the thread
            message = client.beta.threads.messages.create(
                thread_id=thread.id,
                role="user",
                content=user_input
            )

            # Run the assistant to process the message
            run = client.beta.threads.runs.create(
                thread_id=thread.id,
                assistant_id=assistant.id,
                # Can pass additional instructions for this specific run if needed
                # instructions="Please be especially polite."
            )
            print(f"\nRunning assistant... (Run ID: {run.id})")

            # ===>>> LOOP TO WAIT FOR RUN COMPLETION OR ACTION <<<===
            while True: # Keep checking the run status
                run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
                print(f"Status: {run.status}", end='\r')

                if run.status == 'completed':
                    print(f"Status: {run.status}      ") # Clear status line
                    # Get messages added by the assistant
                    messages = client.beta.threads.messages.list(thread_id=thread.id, order="asc", after=message.id) # Get only new messages
                    for msg in messages.data:
                        if msg.role == "assistant":
                            # Extract the response text
                            for content_block in msg.content:
                                if content_block.type == 'text':
                                    print(f"\nAssistant: {content_block.text.value}")
                    break # Exit the inner while loop, wait for next user input

                elif run.status == 'requires_action':
                    print(f"Status: {run.status}      ") # Clear status line
                    print("\nAssistant requires function calls...")
                    tool_outputs = [] # List to store function call results

                    # Assistant might request multiple function calls at once
                    for tool_call in run.required_action.submit_tool_outputs.tool_calls:
                        function_name = tool_call.function.name
                        function_args = json.loads(tool_call.function.arguments) # Arguments come as JSON string

                        print(f"  - Calling function: {function_name}")
                        print(f"    Arguments: {function_args}")

                        # Find the function in our dictionary
                        if function_name in available_functions:
                            function_to_call = available_functions[function_name]
                            try:
                                # Call the actual function with unpacked arguments
                                function_response = function_to_call(**function_args)
                                # Print the beginning of the result
                                print(f"    Result: {str(function_response)[:200]}..." if len(str(function_response)) > 200 else f"    Result: {function_response}")

                                # Add result to the list for sending back to the assistant
                                tool_outputs.append({
                                    "tool_call_id": tool_call.id,
                                    "output": str(function_response), # Result must be a string
                                })
                            except Exception as e:
                                print(f"    Error executing function {function_name}: {e}")
                                # Inform the assistant about the error
                                tool_outputs.append({
                                    "tool_call_id": tool_call.id,
                                    "output": json.dumps({"error": f"Error executing function {function_name}: {e}"})
                                })
                        else:
                            print(f"    Error: Function '{function_name}' not found!")
                            tool_outputs.append({
                                "tool_call_id": tool_call.id,
                                "output": json.dumps({"error": f"Function '{function_name}' not defined."})
                            })

                    # Submit the results back to the assistant
                    if tool_outputs:
                        try:
                            run = client.beta.threads.runs.submit_tool_outputs(
                                thread_id=thread.id,
                                run_id=run.id,
                                tool_outputs=tool_outputs
                            )
                            print("\nSubmitting function results...")
                            # Continue the inner while loop to wait for the next status
                        except Exception as e:
                            print(f"Error submitting tool outputs: {e}")
                            break # Exit the inner while loop on submission error
                    else:
                        print("No tool outputs generated.")
                        break # Exit the inner while loop if no tools to submit

                elif run.status in ['queued', 'in_progress', 'cancelling']:
                    time.sleep(1) # Wait before checking status again
                    continue # Continue the inner while loop

                elif run.status == 'failed':
                    print(f"Status: {run.status}      ") # Clear status line
                    print(f"\nRun failed: {run.last_error.message if run.last_error else 'Unknown error'}")
                    break # Exit the inner while loop
                elif run.status == 'cancelled':
                    print(f"Status: {run.status}      ") # Clear status line
                    print("\nRun cancelled.")
                    break # Exit the inner while loop
                elif run.status == 'expired':
                    print(f"Status: {run.status}      ") # Clear status line
                    print("\nRun expired.")
                    break # Exit the inner while loop
                else:
                    print(f"Status: {run.status}      ") # Clear status line
                    print(f"\nUnexpected run status: {run.status}")
                    break # Exit the inner while loop
            # ===>>> END OF INNER WHILE LOOP <<<===

            # Check if the inner loop was broken due to failure/cancellation/etc.
            if run.status not in ['completed', 'requires_action']: # If run didn't complete normally or need more action
                 break # Exit the main outer while loop

        except KeyboardInterrupt:
            print("\nInteraction interrupted by user.")
            # Optionally cancel the run if it's in progress
            try:
                if 'run' in locals() and run.status in ['queued', 'in_progress']:
                    client.beta.threads.runs.cancel(thread_id=thread.id, run_id=run.id)
                    print("Attempted to cancel the current run.")
            except Exception as cancel_e:
                print(f"Error cancelling run: {cancel_e}")
            break
        except Exception as e:
            print(f"\nAn unexpected error occurred: {e}")
            # Log the error or handle it as needed
            break # Exit loop on unexpected error


print("\nInteraction loop function defined. Uncomment 'run_assistant_interaction()' at the end to start.")




Interaction loop function defined. Uncomment 'run_assistant_interaction()' at the end to start.


In [20]:
# --- Start the interaction ---
# Call this function to start chatting with the assistant
run_assistant_interaction()



New thread created with ID: thread_O74Du6Jkwrz9wDsnN15NfhSE

--- Weather Recipe Guru Ready --- 
Enter your message or 'exit' to finish.

You: Suggest a healthy dinner suitable for the current weather in Dublin.

Running assistant... (Run ID: run_oE91dTEaYx6OyhWVolx5WlZ1)
Status: completed      

Assistant: Hello! I'd be happy to help you with that. First, could you please share your name and confirm that you're currently in Dublin?

You: Arsenii, yeah in Dublin rn 

Running assistant... (Run ID: run_z48GHQvWwIwvkMBJInBNPm5X)
Status: requires_action      

Assistant requires function calls...
  - Calling function: get_user_profile
    Arguments: {'user_id': 'Arsenii'}
    Result: {"user_id": "Arsenii", "preferences": {"likes": ["italian"], "dislikes": ["spicy"]}, "allergies": ["shellfish"]}
  - Calling function: get_weather
    Arguments: {'city': 'Dublin'}

[Function] Requesting weather for city: Dublin
[Function] Weather received: {'temperature': 18.57, 'description': 'clear sky'}
  

In [17]:
# === File Search (Vector Stores via REST API) ===

import openai
import os
import json
import time
import requests # Will be used for Vector Store operations

# --- Check OpenAI Version ---
try:
    print(f"Using OpenAI version: {openai.__version__}")
except NameError:
    print("Error: openai module not imported?")
    import openai
    print(f"Using OpenAI version (after import): {openai.__version__}")

# --- Create RAG file with Cooking Tips ---
rag_file_content_tips = """## Basic Cooking Techniques & Tips

### How to Dice an Onion
1. Cut the onion in half lengthwise (pole to pole).
2. Peel off the skin.
3. Place one half flat-side down. Make several horizontal cuts towards the root, but not all the way through it.
4. Make several vertical cuts lengthwise, again not cutting through the root.
5. Finally, cut crosswise to produce diced pieces. The root end holds it together.

### How to Boil an Egg
- **Soft-boiled:** Place eggs in boiling water. Boil for 4-6 minutes. Plunge into ice water.
- **Hard-boiled:** Place eggs in cold water, bring to a boil. Once boiling, turn off heat, cover, and let sit for 10-12 minutes. Plunge into ice water.

### Common Ingredient Substitutions
*   **Buttermilk:** For 1 cup, use 1 cup milk + 1 tablespoon lemon juice or white vinegar. Let stand for 5 minutes.
*   **Sour Cream:** Plain yogurt or Greek yogurt can often be substituted 1:1.
*   **Vegetable Oil (in baking):** Melted butter (1:1), applesauce (1:1, may affect moisture), or mashed banana (1:1).
*   **Self-Rising Flour:** For 1 cup, use 1 cup all-purpose flour + 1.5 teaspoons baking powder + 0.25 teaspoon salt.

### Food Safety Reminders
*   Always wash hands before and after handling raw meat.
*   Use separate cutting boards for raw meat and produce.
*   Cook chicken to an internal temperature of 74°C (165°F).
*   Refrigerate leftovers promptly.
"""

# Define the new file path
new_rag_file_path = "/home/ubuntu/cooking_tips_rag.txt"

# Make sure the directory exists (important in some environments)
os.makedirs(os.path.dirname(new_rag_file_path), exist_ok=True)

# Write the content to the file
with open(new_rag_file_path, "w", encoding="utf-8") as f:
    f.write(rag_file_content_tips)

print(f"File for RAG '{new_rag_file_path}' created/overwritten.")

# --- Configuration ---
ASSISTANT_ID = None
if 'assistant' in globals() and assistant is not None:
    ASSISTANT_ID = assistant.id
    print(f"Using Assistant ID from cell 5: {ASSISTANT_ID}")
else:
    print("WARNING: Could not find Assistant ID from cell 5. Set ASSISTANT_ID manually!")
    # ASSISTANT_ID = "asst_..." # Insert ID here if needed

FILE_PATH = "/home/ubuntu/cooking_tips_rag.txt"
VECTOR_STORE_NAME = "Seasonal Produce Ireland Store REST"

# --- REST API Helper Functions --- #

def rest_api_request(method, url, api_key, payload=None):
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "OpenAI-Beta": "assistants=v2" # Key header for v2
    }
    try:
        if method.upper() == 'POST':
            response = requests.post(url, headers=headers, json=payload)
        elif method.upper() == 'GET':
            response = requests.get(url, headers=headers)
        elif method.upper() == 'DELETE':
            response = requests.delete(url, headers=headers)
        else:
            raise ValueError(f"Unsupported HTTP method: {method}")

        # Attempt to decode JSON if there is a response body
        response_data = None
        if response.text:
            try:
                response_data = response.json()
            except json.JSONDecodeError:
                print(f"   [REST] Warning: Non-JSON response received (Status: {response.status_code}): {response.text[:500]}...")
                response_data = {"error_text": response.text}

        return response.status_code, response_data

    except requests.exceptions.RequestException as e:
        print(f"   [REST] Network Error: {e}")
        return 500, {"error": str(e)}
    except Exception as e:
        print(f"   [REST] Unexpected Error: {e}")
        return 500, {"error": str(e)}

# --- Check API Key and Assistant ID Prerequisites ---
def check_prerequisites_rest():
    if 'OPENAI_API_KEY' not in globals() or not OPENAI_API_KEY:
        print("Error: OPENAI_API_KEY variable not found. Please run cell 2.")
        return False
    if not ASSISTANT_ID:
         print("Error: ASSISTANT_ID is not defined. Check the beginning of this cell or the output of cell 5.")
         return False
    if not os.path.exists(FILE_PATH):
        print(f"Error: RAG file not found at path: {FILE_PATH}")
        return False
    return True

# --- Main Logic for Updating with Vector Store via REST API ---
def setup_file_search_via_rest():
    if not check_prerequisites_rest():
        return

    local_client = None
    uploaded_file_id = None
    vector_store_id = None
    api_key = OPENAI_API_KEY

    try:
        # Use SDK to upload the file, as that worked
        print("Creating OpenAI client (for file upload)...")
        local_client = openai.OpenAI(api_key=api_key)
        print("Client created.")

        # --- Step 1: Upload File (via SDK) ---
        print(f"1. Uploading file {FILE_PATH} via SDK...")
        with open(FILE_PATH, "rb") as file_to_upload:
            file_object = local_client.files.create(file=file_to_upload, purpose='assistants')
        uploaded_file_id = file_object.id
        print(f"   File uploaded successfully, File ID: {uploaded_file_id}")

        # --- Step 2: Create Vector Store (via REST API) --- #
        print(f"2. Creating Vector Store '{VECTOR_STORE_NAME}' via REST API...")
        vs_create_url = "https://api.openai.com/v1/vector_stores"
        vs_payload = {"name": VECTOR_STORE_NAME}
        status_code, vs_data = rest_api_request('POST', vs_create_url, api_key, payload=vs_payload)

        if status_code == 200 and vs_data and 'id' in vs_data:
            vector_store_id = vs_data['id']
            print(f"   Vector Store created, ID: {vector_store_id}")
        else:
            print(f"   Error creating Vector Store (Status: {status_code}). Response:")
            print(json.dumps(vs_data, indent=2))
            raise Exception("Failed to create Vector Store via REST API.")

        # --- Step 3: Add File to Vector Store (via REST API) --- #
        print(f"3. Adding file {uploaded_file_id} to Vector Store {vector_store_id} via REST API...")
        vs_add_file_url = f"https://api.openai.com/v1/vector_stores/{vector_store_id}/files"
        vs_add_payload = {"file_id": uploaded_file_id}
        status_code, vs_file_data = rest_api_request('POST', vs_add_file_url, api_key, payload=vs_add_payload)

        if status_code == 200 and vs_file_data and 'id' in vs_file_data:
            vs_file_id = vs_file_data['id'] # This is the ID of the file *within* the VS
            print(f"   File {vs_file_id} added to Vector Store. Status: {vs_file_data.get('status', 'N/A')}")
        else:
            print(f"   Error adding file to Vector Store (Status: {status_code}). Response:")
            print(json.dumps(vs_file_data, indent=2))
            raise Exception("Failed to add file to Vector Store via REST API.")

        # --- Step 4: Wait for File Processing in Vector Store (via REST API) --- #
        print(f"4. Waiting for file {vs_file_id} processing in Vector Store (status 'completed')...")
        vs_file_status_url = f"https://api.openai.com/v1/vector_stores/{vector_store_id}/files/{vs_file_id}"
        polling_interval = 5
        max_wait_time = 180
        start_time = time.time()
        while True:
            status_code, file_status_data = rest_api_request('GET', vs_file_status_url, api_key)
            current_status = file_status_data.get('status', 'unknown') if file_status_data else 'error'
            print(f"   Current status of file in VS: {current_status}", end='\r')

            if current_status == 'completed':
                print("\n   File processed successfully in Vector Store.")
                break
            elif current_status in ['failed', 'cancelled']:
                print(f"\n   Error processing file in Vector Store. Status: {current_status}")
                last_error = file_status_data.get('last_error', None)
                if last_error:
                    print(f"      Error: {last_error.get('code', 'N/A')} - {last_error.get('message', 'N/A')}")
                raise Exception(f"Error processing file in Vector Store: {current_status}")
            elif status_code != 200:
                 print(f"\n   Error checking file status (Status: {status_code}). Response:")
                 print(json.dumps(file_status_data, indent=2))
                 raise Exception("Failed to check file status in Vector Store.")

            if time.time() - start_time > max_wait_time:
                 print("\n   Timeout exceeded waiting for file processing in Vector Store.")
                 raise TimeoutError("File was not processed in Vector Store within the allowed time.")

            time.sleep(polling_interval)

        # --- Step 5: Update Assistant (via SDK if supports tool_resources, else REST) --- #
        print(f"5. Updating assistant {ASSISTANT_ID} to use File Search and Vector Store {vector_store_id}...")

        tool_resources_payload = {
            "file_search": {
                "vector_store_ids": [vector_store_id]
            }
        }

        # Attempt update via SDK
        try:
            print("   Attempting update via SDK...")
            assistant_details = local_client.beta.assistants.retrieve(ASSISTANT_ID)
            current_tools = [tool.to_dict() for tool in assistant_details.tools] if assistant_details.tools else []
            updated_tools = list(current_tools)
            if not any(tool.get('type') == 'file_search' for tool in updated_tools):
                updated_tools.append({"type": "file_search"})
                print("      'file_search' tool added.")

            updated_assistant = local_client.beta.assistants.update(
                assistant_id=ASSISTANT_ID,
                tools=updated_tools,
                tool_resources=tool_resources_payload
            )
            print(f"   Assistant '{updated_assistant.name}' updated successfully via SDK!")
            update_success = True
        except (TypeError, AttributeError) as sdk_err:
            print(f"   SDK {openai.__version__} does not support 'tool_resources'. Error: {sdk_err}")
            print("   Falling back to update via REST API...")
            update_success = False
        except Exception as sdk_update_err:
             print(f"   Error updating assistant via SDK: {sdk_update_err}")
             update_success = False

        # If SDK failed, use REST
        if not update_success:
            assistant_update_url = f"https://api.openai.com/v1/assistants/{ASSISTANT_ID}"
            # Get current assistant data via REST for merging
            print("      Getting current assistant data via REST...")
            get_status, current_assistant_data = rest_api_request('GET', assistant_update_url, api_key)
            if get_status != 200 or not current_assistant_data:
                 print(f"      Failed to get current assistant data (Status: {get_status}). Response:")
                 print(json.dumps(current_assistant_data, indent=2))
                 raise Exception("Failed to get assistant data for update via REST.")

            # Form payload for update
            update_payload = {
                "model": current_assistant_data.get("model"), # Need to pass model
                "instructions": current_assistant_data.get("instructions"),
                "name": current_assistant_data.get("name"),
                "tools": current_assistant_data.get("tools", []), # Take current tools
                "tool_resources": tool_resources_payload # Add our resources
            }
            # Add file_search to tools if not present
            if not any(tool.get('type') == 'file_search' for tool in update_payload["tools"]):
                 update_payload["tools"].append({"type": "file_search"})
                 print("      'file_search' tool added to payload.")

            print("      Sending assistant update request via REST...")
            update_status, update_response_data = rest_api_request('POST', assistant_update_url, api_key, payload=update_payload)

            if update_status == 200:
                print(f"   Assistant '{update_response_data.get('name', ASSISTANT_ID)}' updated successfully via REST API!")
            else:
                print(f"   Error updating assistant via REST API (Status: {update_status}). Response:")
                print(json.dumps(update_response_data, indent=2))
                raise Exception("Failed to update assistant via REST API.")

        # --- Final Message --- #
        print(f"\nSUCCESS! File Search (Retrieval) configured via Vector Store {vector_store_id} using REST API.")
        print("\nYou can now return to cell 6 and test File Search.")
        print("Try asking: 'What vegetables are in season in Ireland in May?'")

    except Exception as e:
        print(f"\nAn error occurred during File Search setup: {e}")
        import traceback
        traceback.print_exc()
        # --- Attempt resource cleanup on error --- #
        print("\n--- Attempting resource cleanup --- ")
        if vector_store_id:
            try:
                print(f"   Deleting Vector Store {vector_store_id}...")
                delete_vs_url = f"https://api.openai.com/v1/vector_stores/{vector_store_id}"
                status, data = rest_api_request('DELETE', delete_vs_url, api_key)
                if status == 200 and data and data.get('deleted'):
                    print(f"      Vector Store {vector_store_id} deleted.")
                else:
                    print(f"      Failed to delete Vector Store {vector_store_id} (Status: {status}). Response: {data}")
            except Exception as delete_vs_err:
                print(f"      Error deleting Vector Store: {delete_vs_err}")
        # Delete file uploaded via SDK using SDK
        if uploaded_file_id and local_client:
            try:
                print(f"   Deleting file {uploaded_file_id}...")
                local_client.files.delete(uploaded_file_id)
                print(f"      File {uploaded_file_id} deleted.")
            except Exception as delete_file_err:
                print(f"      Error deleting file: {delete_file_err}")
        elif uploaded_file_id:
             print(f"   Failed to delete file {uploaded_file_id} (SDK client not initialized).")

# --- Run Setup --- #
setup_file_search_via_rest()



Using OpenAI version: 1.76.0
File for RAG '/home/ubuntu/cooking_tips_rag.txt' created/overwritten.
Using Assistant ID from cell 5: asst_36JdysTVWsHUJHfz3DzIXloq
Creating OpenAI client (for file upload)...
Client created.
1. Uploading file /home/ubuntu/cooking_tips_rag.txt via SDK...
   File uploaded successfully, File ID: file-BDDUEth3tHeLqBMgWPpadc
2. Creating Vector Store 'Seasonal Produce Ireland Store REST' via REST API...
   Vector Store created, ID: vs_6817b26278d48191875d10b3c67c3853
3. Adding file file-BDDUEth3tHeLqBMgWPpadc to Vector Store vs_6817b26278d48191875d10b3c67c3853 via REST API...
   File file-BDDUEth3tHeLqBMgWPpadc added to Vector Store. Status: in_progress
4. Waiting for file file-BDDUEth3tHeLqBMgWPpadc processing in Vector Store (status 'completed')...
   Current status of file in VS: completed
   File processed successfully in Vector Store.
5. Updating assistant asst_36JdysTVWsHUJHfz3DzIXloq to use File Search and Vector Store vs_6817b26278d48191875d10b3c67c3853.

In [3]:
import openai


print("OpenAI version:", openai.__version__)


try:
    from openai.resources import assistants_files
except ImportError as e:
    print(" ImportError:", e)


OpenAI version: 1.76.0
 ImportError: cannot import name 'assistants_files' from 'openai.resources' (/usr/local/lib/python3.11/dist-packages/openai/resources/__init__.py)


In [14]:
!sudo apt-get update && sudo apt-get install -y sqlite3


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [77.5 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,715 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe a

In [15]:
!sqlite3 -header -column "/content/drive/MyDrive/ColabData/recipe_assistant_db.sqlite" "SELECT * FROM user_profiles;"


user_id  preferences_json                               allergies_json
-------  ---------------------------------------------  --------------
Arsenii  {"likes": ["italian"], "dislikes": ["spicy"]}  ["shellfish"] 


In [ ]:
!apt-get update && apt-get install -y sqlite3


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,659 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,200 kB]
Get:13 http://archive.ubuntu.com/ubuntu jamm

In [19]:
# === Gradio Chat Interface Cell (Fixed v7) ===

print("Installing Gradio...")
!pip install gradio -q

print("Importing necessary modules for Gradio UI...")
import gradio as gr
import openai
import time
import json
import os
import requests
import sqlite3 # Needed for profile functions

# --- Ensure previous cells have run ---
# Check for necessary variables from previous cells
required_vars = [
    'client', 'ASSISTANT_ID', 'OPENAI_API_KEY',
    'SPOONACULAR_API_KEY', 'OPENWEATHERMAP_API_KEY',
    'DB_PATH' # From cell 2/3 for SQLite
]
missing_vars = [var for var in required_vars if var not in globals()]

if missing_vars:
    print(f"\nERROR: The following required variables are missing: {', '.join(missing_vars)}")
    print("Please ensure you have run the previous cells (especially 2, 3, 5, and 19 if using File Search) successfully.")
    # Stop execution if prerequisites are not met
    raise NameError(f"Missing required variables: {', '.join(missing_vars)}")
else:
    print("\nAll necessary variables found. Proceeding with Gradio setup.")

# --- Re-define necessary helper functions (or ensure they are globally accessible) ---
# Assuming functions from cells 3 and 6 (get_db_connection, init_db, get_user_profile, etc.,
# get_weather, find_recipes, get_recipe_details) are available globally.

# Make sure DB is initialized (redundant if cell 3 ran, but safe)
try:
    init_db()
except NameError:
    print("Warning: init_db() function not found globally. Assuming DB is initialized.")
except Exception as db_init_err:
    print(f"Warning: Error during redundant DB initialization: {db_init_err}")

# Dictionary mapping function names to implementations (must be globally accessible)
available_functions = {
    "get_weather": get_weather,
    "find_recipes": find_recipes,
    "get_recipe_details": get_recipe_details,
    "get_user_profile": get_user_profile,
    "save_user_preference": save_user_preference,
    "save_user_allergy": save_user_allergy,
}

# --- Core Logic for Gradio Chatbot ---

def process_run(thread_id, run_id):
    """Processes a run, handles actions, and returns the final assistant message string."""
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"[Gradio Backend] Run Status: {run.status}") # Debug output

        if run.status == 'completed':
            messages = client.beta.threads.messages.list(thread_id=thread_id, order="desc", limit=1) # Get latest message
            assistant_message = "" # Initialize
            if messages.data and messages.data[0].role == "assistant":
                for content_block in messages.data[0].content:
                    if content_block.type == 'text':
                        assistant_message += content_block.text.value + "\n"
            return assistant_message.strip() if assistant_message else "Assistant did not provide a text response."

        elif run.status == 'requires_action':
            print("[Gradio Backend] Assistant requires function calls...")
            tool_outputs = []
            for tool_call in run.required_action.submit_tool_outputs.tool_calls:
                function_name = tool_call.function.name
                try:
                    function_args = json.loads(tool_call.function.arguments)
                except json.JSONDecodeError:
                     print(f"[Gradio Backend] Error: Invalid JSON arguments from assistant for {function_name}: {tool_call.function.arguments}")
                     tool_outputs.append({"tool_call_id": tool_call.id, "output": json.dumps({"error": "Invalid JSON arguments received"})})
                     continue # Skip this tool call

                print(f"  - Calling function: {function_name}")
                print(f"    Arguments: {function_args}")

                if function_name in available_functions:
                    function_to_call = available_functions[function_name]
                    try:
                        function_response = function_to_call(**function_args)
                        output_str = str(function_response)
                        print(f"    Result: {output_str[:200]}..." if len(output_str) > 200 else f"    Result: {output_str}")
                        tool_outputs.append({"tool_call_id": tool_call.id, "output": output_str})
                    except Exception as e:
                        print(f"    Error executing function {function_name}: {e}")
                        tool_outputs.append({"tool_call_id": tool_call.id, "output": json.dumps({"error": f"Error in {function_name}: {e}"})})
                else:
                    print(f"    Error: Function '{function_name}' not found!")
                    tool_outputs.append({"tool_call_id": tool_call.id, "output": json.dumps({"error": f"Function '{function_name}' not defined."})})

            if tool_outputs:
                try:
                    client.beta.threads.runs.submit_tool_outputs(
                        thread_id=thread_id,
                        run_id=run.id,
                        tool_outputs=tool_outputs
                    )
                    print("[Gradio Backend] Submitted function results.")
                    # Continue the loop to wait for the next status
                except Exception as e:
                    print(f"[Gradio Backend] Error submitting tool outputs: {e}")
                    return f"Error submitting function results: {e}" # Return error message
            else:
                print("[Gradio Backend] No tool outputs generated despite requires_action status.")
                return "Error: Assistant required action but no function calls were processed." # Return error message

        elif run.status in ['queued', 'in_progress', 'cancelling']:
            time.sleep(1) # Wait before checking again

        elif run.status in ['failed', 'cancelled', 'expired']:
            error_message = f"Run {run.status.upper()}."
            if run.last_error:
                error_message += f" Reason: {run.last_error.message}"
            print(f"[Gradio Backend] {error_message}")
            return error_message # Return error message
        else:
            print(f"[Gradio Backend] Unexpected run status: {run.status}")
            return f"Unexpected error: Run status is {run.status}" # Return error message

def assistant_chat(message, history, thread_id_state):
    """Handles a single turn of the chat. Returns tuple: (updated_thread_id, assistant_response_string)."""
    print(f"\n[Gradio Backend] Received message: '{message}'")
    print(f"[Gradio Backend] Current Thread ID State: {thread_id_state}")
    # History is managed by Gradio ChatInterface itself when type="messages", we don't need to process it here

    # Get or create thread ID
    thread_id = thread_id_state
    if not thread_id:
        try:
            thread = client.beta.threads.create()
            thread_id = thread.id
            print(f"[Gradio Backend] New thread created: {thread_id}")
        except Exception as e:
            print(f"[Gradio Backend] Error creating thread: {e}")
            # Return current state and error message string
            return thread_id_state, f"Error creating conversation thread: {e}"

    # Add user message to the thread
    try:
        client.beta.threads.messages.create(
            thread_id=thread_id,
            role="user",
            content=message
        )
        print(f"[Gradio Backend] User message added to thread {thread_id}")
    except Exception as e:
        print(f"[Gradio Backend] Error adding message to thread: {e}")
        # Return updated thread_id and error message string
        return thread_id, f"Error sending message: {e}"

    # Run the assistant
    try:
        run = client.beta.threads.runs.create(
            thread_id=thread_id,
            assistant_id=ASSISTANT_ID
        )
        print(f"[Gradio Backend] Assistant run created: {run.id}")

        # Process the run and get the final response string
        assistant_response = process_run(thread_id, run.id)
        print(f"[Gradio Backend] Assistant response string: {assistant_response}")

        # Return the updated thread_id and the assistant's response string
        return thread_id, assistant_response

    except Exception as e:
        print(f"[Gradio Backend] Error running assistant or processing run: {e}")
        # Return updated thread_id and error message string
        return thread_id, f"Error during assistant processing: {e}"

# --- Setup Gradio Interface --- #
print("\nSetting up Gradio Chat Interface...")

# Use gr.State to manage the thread_id across calls within a session
thread_state = gr.State(None)

# Wrapper function for Gradio ChatInterface
def stateful_chat_wrapper(message, history, current_thread_id):
    """Takes message, history, and state. Returns tuple: (assistant_response_string, updated_thread_id)."""
    # If history is None (first turn), initialize it as an empty list
    if history is None:
        history = []
    print(f"[Gradio Backend] History received (type={type(history)}): {history}") # Debug history

    # Call the main logic function which returns (updated_thread_id, response_string)
    final_thread_id, final_response = assistant_chat(message, history, current_thread_id)

    # *** IMPORTANT: Return the response string AND the updated state ***
    # The history is automatically updated by ChatInterface when type="messages".
    # The first return value goes to the chatbot output.
    # The second return value goes to the state output (additional_outputs[0]).
    print(f"[Gradio Backend] Returning response string: {final_response}")
    print(f"[Gradio Backend] Returning final thread ID: {final_thread_id}")

    return final_response, final_thread_id


# Define the Gradio components explicitly for clarity
# *** CHANGE: Explicitly set type="messages" ***
chatbot = gr.Chatbot(label="Weather Recipe Guru", height=600, type="messages")
textbox = gr.Textbox(placeholder="Ask me for recipe ideas based on the weather!", container=False, scale=7)

# Create the ChatInterface
iface = gr.ChatInterface(
    fn=stateful_chat_wrapper,
    chatbot=chatbot,
    textbox=textbox,
    title="Weather Recipe Guru Assistant",
    description="Chat with the assistant to get recipe suggestions based on weather, preferences, and allergies.",
    theme="soft",
    examples=[["Hello"], ["What can I make for dinner in London?"], ["Suggest a vegetarian soup, I don't like mushrooms"]],
    cache_examples=False,
    # Pass the state variable as an additional input and output
    # Inputs: message (from textbox), history (from chatbot), current_thread_id (from state)
    # Outputs:
    # 1. The first return value of fn (final_response) goes to chatbot.
    # 2. The second return value of fn (final_thread_id) goes to additional_outputs[0] (thread_state).
    additional_inputs=[thread_state],
    additional_outputs=[thread_state]
)

# --- Launch Gradio Interface --- #
print("\nLaunching Gradio Interface...")
iface.launch(inline=True, debug=True) # debug=True shows more logs

print("\nGradio Interface launched. Interact with the chat window above.")



Installing Gradio...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 143.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.2 MB/s eta 0:00:00
Importing necessary modules for Gradio UI...

All necessary variables found. Proceeding with Gradio setup.
Database initialized successfully.

Setting up Gradio Chat Interface...


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:321: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'messages', will be used.
  warnings.warn(



Launching Gradio Interface...
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a1f9cb72f470ea7a0b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[Gradio Backend] History received (type=<class 'list'>): []

[Gradio Backend] Received message: 'Hello'
[Gradio Backend] Current Thread ID State: None
[Gradio Backend] New thread created: thread_GTDiTiMrgYV9370CqFkyjyJB
[Gradio Backend] User message added to thread thread_GTDiTiMrgYV9370CqFkyjyJB
[Gradio Backend] Assistant run created: run_w9826n0D4axBTX1RbDHMbMio
[Gradio Backend] Run Status: queued
[Gradio Backend] Run Status: in_progress
[Gradio Backend] Run Status: completed
[Gradio Backend] Assistant response string: Hi there! How can I assist you today? If you don't mind sharing, what's your name and which city are you in? This will help me tailor my suggestions for you!
[Gradio Backend] Returning response string: Hi there! How can I assist you today? If you don't mind sharing, what's your name and which city are you in? This will help me tailor my suggestions for you!
[Gradio Backend] Returning final thread ID: thread_GTDiTiMrgYV9370CqFkyjyJB
[Gradio Backend] History received (ty